# Data Preparation

## Pre-processing

In [105]:
import numpy as np
import pandas as pd
import json
import re
from sklearn.preprocessing import LabelEncoder, LabelBinarizer


df = pd.read_csv('movies_metadata.csv')

print(len(df),'items in unprocessed dataframe')
df = df[df.status=='Released']
df = df[df.original_language=='en']
df = df.drop(columns=['adult','belongs_to_collection','homepage','imdb_id','original_language','overview',
                     'poster_path','release_date','spoken_languages',
                     'status','tagline','video','original_title'])

def contains_us(row):
    return 'US' in row

def get_genres(row):
    s = re.findall("'name': '(.*?)'",row)
    if len(s) > 0:
        return s
    else:
        return None

def get_prod_companies(row):

    s = re.findall("{'name': '(.*?)'",row)
    '''
    if len(s) > 0:
        return s
    else:
        return None
    '''
    if len(s) > 1:
        return 'Collaboration'
    elif len(s) == 1:
        return s[0]
    else:
        return None

def discretize_runtime(row):
    if row < 40:
        return 'Short Film'
    elif row < 80:
        return 'Less than Feature Film'
    elif row < 130:
        return 'Below Average Feature Film'
    else:
        return 'Above Average Feature Film'

df['made_in_us'] = df.production_countries.apply(contains_us)
df = df.drop(columns=['production_countries'])
df['production_companies'] = df.production_companies.apply(get_prod_companies)

df['genres'] = df.genres.apply(get_genres)

df = df.dropna()
enc = LabelEncoder()
df['production_companies_int'] = enc.fit_transform(df.production_companies)

df['runtime_category'] = df.runtime.apply(discretize_runtime)
df['runtime_category_int'] = enc.fit_transform(df.runtime_category)

df['budget'] = df.budget.apply(float)
df['popularity'] = df.popularity.apply(float)
df['id'] = df.id.apply(int)


df['genres_int'] = enc.fit_transform(df.genres)
avg = df.vote_average.mean()
df['above_average_vote'] = df.vote_average > avg

print(max(df.production_companies_int)+1,'classes for production_companies')
print(max(df.genres_int)+1,'classes for genres')

print(len(df),'items in processed dataframe\n')
print(list(df))
print(df.dtypes)

categorical_headers_ints = ['production_companies_int','genres_int','runtime_category_int']
numeric_headers = ['budget','popularity','revenue','runtime','vote_average','vote_count']

c:\users\momin\envs\ml_lab1\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


45466 items in unprocessed dataframe
4572 classes for production_companies
2970 classes for genres
23298 items in processed dataframe

['budget', 'genres', 'id', 'popularity', 'production_companies', 'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'made_in_us', 'production_companies_int', 'runtime_category', 'runtime_category_int', 'genres_int', 'above_average_vote']
budget                      float64
genres                       object
id                            int64
popularity                  float64
production_companies         object
revenue                     float64
runtime                     float64
title                        object
vote_average                float64
vote_count                  float64
made_in_us                     bool
production_companies_int      int64
runtime_category             object
runtime_category_int          int64
genres_int                    int64
above_average_vote             bool
dtype: object


In [106]:
print(df.vote_average.quantile([.25,.5,.75,.9,.99,1]))
print(len(df[df.vote_count>0]))

0.25     5.1
0.50     6.0
0.75     6.7
0.90     7.3
0.99     8.3
1.00    10.0
Name: vote_average, dtype: float64
22449


In this lab, we work with the Kaggle Movies Dataset. This dataset contains metadata on 45,466 different movies released on or before July 2017. The dataset includes information about each movie such as the budget, the genres, the runtime, the revenue, and popularity scores based on data from the research group GroupLens.

In preprocessing the data, we first remove the features we will not be using for classification. Then, we create a few new columns.

First, we create a column called made_in_us. This is a modification of the original column production_countries, which contains a list of the countries each movie was produced in. Since there were a large number of such countries, we chose to represent that column as a binary value of if this movie was produced in the United States or in a different country.

Next, we create a column called production_companies_int. This is a modification of the original column production_companies, which contains a list of the production companies that created each movie. We take this list of production companies and represent collaborations between multiple production companies as the class "collaboration" - we do this because there is a large number of combinations otherwise. Then, we encode the production company as an integer using sklearn's LabelEncoder class.

Next, we create a column called genres_int. This is a modification of the original column genres, which contains a list of the genres each movie belongs to. We take this list of genres and encode it using sklearn's LabelEncoder class. In the case that there are multiple genres for a movie, such as a romantic comedy, we represent this combination as its own class, unlike with the production companies. The rationale behind this choice is that there is a much larger number of production companies than genres - we can represent all combinations of genres with 2,970 classes. 

Next, we create a column called above_average_vote. This is a modification of the original column average_vote, which is the average of all votes given to this movie on a scale of 0-10. This column is the target column; we will try to predict if a movie can score above average.

Finally, we create a column called runtime_category_int. This is a modification of the original column runtime, which is the runtime of the movie in minutes. We discretize the runtime into 4 categories: short film, less than feature film, below average feature film, and above average feature film. We use the following cutoffs: a short film is less than 40 minutes, a movie that is less than a feature film is less than 80 minutes, a feature film below average is less than 130 minutes, and a feature film above average is longer than 130 minutes. 

We derive the runtime of a short film from the rules the Oscar awards establish for the short film category (http://www.oscars.org/sites/oscars/files/90aa_short_films.pdf). We derive the length of a feature film from the Screen Actor's Guild (http://www.sagaftra.org/files/sag/Low_Budget_Ageement_1_5.pdf). We derive the average length of a feature film from a Business Insider study (http://www.businessinsider.com/movies-are-getting-longer-2013-1).

We choose to represent the runtime category as categorical data and not ordinal data. The reason we represent runtime categorically instead of ordinally is because different categories of film are not necessarily similar to one another. For example, below average feature films are closer in length to short films than above average feature films are. However, since short films are often not shown in movie theatres, it is likely that there is an entirely different audience for short films than there is for below average length feature films.

## Crossing Features

### Train Test Split

In [107]:
from sklearn.model_selection import StratifiedKFold
from copy import deepcopy

df_features = ['budget', 'genres', 'id', 'popularity', 'production_companies', 'revenue', 
               'runtime', 'title', 'vote_average', 'vote_count', 'made_in_us', 'production_companies_int', 
               'runtime_category', 'runtime_category_int', 'genres_int']
df_class = ['above_average_vote']

X = df[df_features]
y = df[df_class]

train_list = []
test_list = []

for train_index, test_index in skf.split(X, y):
    train_list.append(train_index)
    test_list.append(test_index)




df_train = deepcopy(df.iloc[train_list[0]])
df_test = deepcopy(df.iloc[test_list[0]])

#print (df_train.head())
#print (df_train.head())

### One Hot Encoding Categorical Features

In [108]:
from sklearn.preprocessing import OneHotEncoder

# now let's encode the integer outputs as one hot encoded labels
ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(df_train[categorical_headers_ints].values)
X_test_ohe = ohe.transform(df_test[categorical_headers_ints].values)

y_train = df_train[df_class].values.astype(np.int)
y_test = df_test[df_class].values.astype(np.int)

# the ohe instance will help us to organize our encoded matrix
print(ohe.feature_indices_)
print(X_train_ohe.shape)
print(type(X_train_ohe), '========THIS IS SPARSE ENCODED=======')

[   0 4572 7542 7546]
(20967, 7202)
<class 'scipy.sparse.csr.csr_matrix'> ========THIS IS SPARSE ENCODED=======


# TODO: Cross the features

In [109]:
# combine the features with two branches

# let's encode the integer outputs as one hot encoded labels
ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(df_train[categorical_headers_ints].values)
X_test_ohe = ohe.transform(df_test[categorical_headers_ints].values)

# and save off the numeric features
X_train_num =  df_train[numeric_headers].values
X_test_num = df_test[numeric_headers].values

# create sparse input branch for ohe
inputsSparse = Input(shape=(X_train_ohe.shape[1],),sparse=True, name='X_ohe')
xSparse = Dense(units=10, activation='relu', name='ohe_1')(inputsSparse)

# create dense input branch for numeric
inputsDense = Input(shape=(X_train_num.shape[1],),sparse=False, name='X_Numeric')
xDense = Dense(units=10, activation='relu',name='num_1')(inputsDense)

x = concatenate([xSparse, xDense], name='concat')
predictions = Dense(1,activation='sigmoid', name='combined')(x)

# This creates a model that includes
# the Input layer and Dense layers
model = Model(inputs=[inputsSparse,inputsDense], outputs=predictions)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_ohe (InputLayer)              (None, 7202)         0                                            
__________________________________________________________________________________________________
X_Numeric (InputLayer)          (None, 6)            0                                            
__________________________________________________________________________________________________
ohe_1 (Dense)                   (None, 10)           72030       X_ohe[0][0]                      
__________________________________________________________________________________________________
num_1 (Dense)                   (None, 10)           70          X_Numeric[0][0]                  
__________________________________________________________________________________________________
concat (Co

In [110]:
%%time

model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit([ X_train_ohe, X_train_num ], # inputs for each branch are a list
          y_train, 
          epochs=10, 
          batch_size=50, 
          verbose=0)

yhat = model.predict([X_test_ohe,
                      X_test_num]) # each branch has an input

yhat = np.round(yhat)
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[ 290  618]
 [  64 1359]] 0.7074217074217074
Wall time: 7.78 s


In [111]:
#insert crossing code

def has_action(row):
    return 'Action' in row

def has_animation(row):
    return 'Animation' in row

action_movies = df[df.genres.apply(has_action)]
animation_movies = df[df.genres.apply(has_animation)]

print(action_movies.runtime.mean())
print(animation_movies.runtime.mean())

100.46123161306407
61.03592233009709


In [112]:
from sklearn import metrics as mt
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model

After preprocessing, we create cross-product features for the wide part of our wide-and-deep network. We cross the values for genres_int and runtime_category_int, as well as the values for genres_int and made_in_us_int.

We decide to cross the values for genres_int and runtime_category_int because films of different genres often have different lengths. For example, action movies have an average runtime of 100.46 minutes, whereas animation movies have an average runtime of 61.04 minutes. Thus, we speculate that a long animation movie will have a different reaction than a long action movie.

Next, we cross the values for genres_int and made_in_us int. We base this feature cross on the idea that different countries may prefer different genres of movie. This hypothesis is supported by independent research from the New York Film Academy (https://www.nyfa.edu/student-resources/12-of-the-most-popular-movie-genres-by-country/) and the American Film Market & Conferences association (http://americanfilmmarket.com/relative-popularity-genres-around-world/).

## Evaluation metric

Our chosen task is a binary classification task: can we predict if a movie will score above or below average when rated by users on a scale of 0-10? We found in our data set that the average of the ~22,000 movies reviewed is a 6.0/10; therefore, a movie that scores above 6.0 is above average, any a movie scoring 6.0 or less is not.

The business case for our prediction task is a movie studio trying to determine if their next movie will score above average or not. We imagine that a movie studio would come to us for our analysis after defining the basic criteria for their movie, such as genre, budget, and approximate runtime. At this point in the production cycle, the firm has not yet invested large amounts of time or money into creating the movie outline. Thus, the movie studio would much rather us reject a good idea than accept a bad idea; a rejected good idea is a waste of the small amount of time put into creating a movie outline, but an accepted bad idea will waste future resources as well as the outline time.

With this use case in mind, we advance precision as our chosen evaluation criteria. Precision, mathematically defined as tp/(tp+fp), is a measure of what percentage of named positives are true positives. In our use case, a precision score of 30% means that, of the movies we said would do better than average, only 30% actually did better than average.

The movie studio would be interested in a high-precision system because this system is good at telling if a movie will be above average. A 90% precision score means that 9 out of 10 movies that we say will succeed do in fact succeed; this goves the movie studio an idea of how likely it is that their movie is a winner.

## Train/test split

For this lab, we will use a 10-fold stratified cross-validation approach for splitting our data into training and testing sets. The reason we use cross-validation instead of shuffle-split is that we worry we do not have enough data. After pre-processing, we are left with ~22,000 movies. This is not an entirely small dataset; however, the feature space is quite large. For example, the movie genre alone can take on 2,970 different values, and the production company 4,572 values. We use 10-folds so that 90% of our data comprises the training set in each fold; this will ensure to the best of our ability that we have enough data.

The reason we use stratified cross-validation is to preserve the class distributions seen in the data; if one class of a feature occurs 10% of the time in the data, it will occur 10% of the time in the training data and 10% of the time in the testing data. We use stratified cross-validation because our data represent the real-world distribution. When applied to real movies, it will also be true that there is large class imbalance in certain features; it is unlikely that in the future a dramatically different percentage of the market will be dominated by Animated-Romantic-Action-Adult films.